# 라이브러리 불러오기

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from data.load_dataset import load_dataset
from data.merge_dataset import merge_dataset
from data.feature_engineering import apply_log_transformation, ClusteringModel
# from model.inference import save_csv
# from model.feature_select import select_features
# from model.data_split import split_features_and_target
# from model.model_train import cv_train, set_model, optuna_train
import argparse
import os
import wandb
import warnings
warnings.filterwarnings("ignore")

# Data Load

In [27]:
# 기존 데이터 불러오기
train_data, test_data, sample_submission, interest_data, subway_data, school_data, park_data = load_dataset()

# 기존 데이터에 새로운 feature들을 병합한 데이터프레임 불러오기
train_data, test_data = merge_dataset(train_data, test_data, interest_data, subway_data, school_data, park_data)

# Data Preprocessing

In [28]:
# 위치 중복도 낮은 행 삭제
groups = train_data.groupby(["latitude", "longitude"])["index"].count()
conditioned_groups_index = groups[(groups >= 2) & (groups <= 5)].index # 이 범위를 파라미터로 조정하는걸로
small_groups = train_data[
    train_data["latitude"].isin(conditioned_groups_index.get_level_values(0)) &
    train_data["longitude"].isin(conditioned_groups_index.get_level_values(1))
]
train_data.drop(small_groups.index, axis=0, inplace=True)

In [29]:
# built_year > 2024 행 삭제
train_data = train_data[train_data["built_year"] < 2024]
train_data.reset_index(drop=True, inplace=True)

# Feature Engineering

In [30]:
#clustering
cluster_data = train_data[["latitude", "longitude"]]
clustering_model = ClusteringModel(cluster_data)
kmeans_model = clustering_model.kmeans_clustering(
    n_clusters = 25,
    train_data = train_data,
    test_data = test_data,
    feature_columns = ["latitude", "longitude"],
    label_column = 'region'
)

In [31]:
# region_mean 병합
region_mean = train_data.groupby('region')['deposit'].mean().reset_index()
region_mean.columns = ['region', 'region_mean']
train_data = train_data.merge(region_mean, on='region', how='left')
test_data = test_data.merge(region_mean, on='region', how='left')

In [32]:
# log 변환
train_data, test_data = apply_log_transformation(train_data, test_data)

#### 가격 Clustering EDA

In [33]:
# sorted_train_data = train_data.sort_values(by="deposit").reset_index(drop=True)
# sorted_train_data["deposit_group"] = sorted_train_data.index // 180000
# train_data = sorted_train_data
# print(train_data.groupby('deposit_group')['deposit'].agg(['min', 'max', 'mean', 'count']))

In [34]:
# 먼저 데이터 정렬 및 인덱스 리셋
sorted_train_data = train_data.sort_values(by="deposit").reset_index(drop=True)
# deposit을 기준으로 그룹을 나눔
# 10,000 미만은 그룹 0, 10,000~100,000 사이는 그룹 1, 100,000 이상은 100,000 단위로 그룹화
def categorize_deposit(deposit):
    if deposit < 10000:
        return 0  # 10,000 미만
    elif deposit <= 100000:
        return 1  # 10,000 ~ 100,000 사이
    elif deposit <= 200000:
        return 2
    elif deposit <= 300000:
        return 3
    elif deposit <= 400000:
        return 4
    elif deposit <= 500000:
        return 5
    elif deposit <= 600000:
        return 6
    else:
        return 7 
# 그룹화 적용
sorted_train_data["deposit_group"] = sorted_train_data["deposit"].apply(categorize_deposit)
# 그룹별 통계 출력
train_data = sorted_train_data
print(train_data.groupby('deposit_group')['deposit'].agg(['min', 'max', 'mean', 'count']))

                    min       max           mean    count
deposit_group                                            
0                 300.0    9990.0    7131.513486    54576
1               10000.0  100000.0   36449.974768  1689931
2              100100.0  200000.0  131001.424749    42717
3              201500.0  300000.0  238188.343877     2466
4              304000.0  400000.0  347309.405941      303
5              405000.0  500000.0  450325.000000       80
6              505000.0  600000.0  553764.705882       34
7              620000.0  950000.0  734527.777778       18


In [35]:
# Feature Select
selected_cols = [
    "log_area_m2",
    "built_year",
    "latitude",
    "longitude",
    "log_subway_distance",
    "log_school_distance",
    "log_park_distance",
    "contract_year_month",
    "contract_day",
    "num_of_subways_within_radius",
    "num_of_parks_within_radius",
    "region",
    "region_mean",
]
X, test_data = train_data[selected_cols], test_data[selected_cols]

# Data Split
y = train_data["deposit_group"]

In [36]:
# from sklearn.feature_selection import SelectKBest, f_regression, f_classif, RFE

# def select_kbest(X, y, target, k=10):
#     """
#     SelectKBest 방법을 사용하여 상위 k개의 특성 선택
#     Args:
#         X (DataFrame): 독립변수
#         y (DataFrame): 종속변수
#         target (str): 종속변수 열 중 실제 사용할 target 열 이름
#         k (int, optional): 선택할 상위 k개 특성의 수 (Defaults to 10)
#     Returns:
#         List[str]: 선택된 상위 k개의 특성의 열 이름 리스트
#     """
#     # SelectKBest 적용
#     selector = SelectKBest(score_func=f_classif, k=k)
#     selector.fit(X, y[target])
#     # 선택된 특성의 열 이름 리스트 반환
#     selected_cols = X.columns[selector.get_support()].tolist()
#     return selected_cols

In [37]:
# selected_cols = select_kbest(X, train_data, "deposit_group")
# selected_cols

In [38]:
print(f"X: {X.columns}")
print(f"test_data: {test_data.columns}")
print(f"y: {y.shape}")

X: Index(['log_area_m2', 'built_year', 'latitude', 'longitude',
       'log_subway_distance', 'log_school_distance', 'log_park_distance',
       'contract_year_month', 'contract_day', 'num_of_subways_within_radius',
       'num_of_parks_within_radius', 'region', 'region_mean'],
      dtype='object')
test_data: Index(['log_area_m2', 'built_year', 'latitude', 'longitude',
       'log_subway_distance', 'log_school_distance', 'log_park_distance',
       'contract_year_month', 'contract_day', 'num_of_subways_within_radius',
       'num_of_parks_within_radius', 'region', 'region_mean'],
      dtype='object')
y: (1790125,)


# Modeling

In [39]:
import xgboost as xgb
from sklearn.metrics import log_loss, mean_absolute_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import optuna
from tqdm import tqdm

In [40]:
run = str(input("run 이름을 입력하세요 :"))
selected_model = str(input("model 명을 입력하세요 (xgb/rf) :"))
opt = bool(input("Optuna 사용 여부를 입력하세요 (뭐라도 입력 시 사용) :"))

wandb.init(
    settings=wandb.Settings(start_method="thread"),
    dir=None,  # 로컬에 로그 저장하지 않음
    entity="remember-us", # team name,
    project="deposit", # project name
    name=run, # run name
    config={
        "User": os.path.basename(os.path.dirname(os.getcwd())) # jupyter는 이렇게
    } # common setting
)

In [41]:
def train(model_name, X, y, params):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

    match model_name:
        case "xgb-cls":
            model = xgb.XGBClassifier(**params, random_state=42, device="cuda", use_label_encoder=False, n_jobs=-1)
            model.fit(X_train, y_train)

            y_pred_proba = model.predict_proba(X_valid)
            score = log_loss(y_valid, y_pred_proba)
            
        case "xgb-reg":
            model = xgb.XGBRegressor(**params, random_state=42, device="cuda", n_jobs=-1)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_valid)
            score = mean_absolute_error(y_valid, y_pred)
        
        case "rf-cls":
            model = RandomForestClassifier(**params, random_state=42, n_jobs=-1)
            model.fit(X_train, y_train)

            y_pred_proba = model.predict_proba(X_valid)
            score = log_loss(y_valid, y_pred_proba)
        
        case "rf-reg":
            model = RandomForestRegressor(**params, random_state=42, n_jobs=-1)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_valid)
            score = mean_absolute_error(y_valid, y_pred)
            
        case _:
            raise ValueError(f"지원하지 않는 모델 이름: {model_name}")
    
    return score

In [42]:
def cv_train(model_name, X, y, params):
    cv = 5
    kfold = KFold(n_splits=cv, shuffle=True, random_state=42)

    score_list = []
    for i, (train_idx, valid_idx) in enumerate(kfold.split(X, y), start=1):
        X_train, y_train = X.loc[train_idx, :], y.iloc[train_idx]
        X_valid, y_valid = X.loc[valid_idx, :], y.iloc[valid_idx]
        
        match model_name:
            case "xgb-cls":
                model = xgb.XGBClassifier(**params, random_state=42, device="cuda", use_label_encoder=False, n_jobs=-1)
                model.fit(X_train, y_train)

                y_pred_proba = model.predict_proba(X_valid)
                fold_score = log_loss(y_valid, y_pred_proba)
                
            case "xgb-reg":
                model = xgb.XGBRegressor(**params, random_state=42, device="cuda", n_jobs=-1)
                model.fit(X_train, y_train)

                y_pred = model.predict(X_valid)
                fold_score = mean_absolute_error(y_valid, y_pred)
            
            case "rf-cls":
                model = RandomForestClassifier(**params, random_state=42, n_jobs=-1)
                model.fit(X_train, y_train)

                y_pred_proba = model.predict_proba(X_valid)
                fold_score = log_loss(y_valid, y_pred_proba)
            
            case "rf-reg":
                model = RandomForestRegressor(**params, random_state=42, n_jobs=-1)
                model.fit(X_train, y_train)

                y_pred = model.predict(X_valid)
                fold_score = mean_absolute_error(y_valid, y_pred)
                
            case _:
                raise ValueError(f"지원하지 않는 모델 이름: {model_name}")
                
        score_list.append(fold_score)
    mean_score = np.mean(score_list)
    return mean_score

In [43]:
def optuna_train(model_name, X, y):
    match model_name:
        case "xgb-cls":
            def objective(trial):
                params = {
                    "n_estimators": trial.suggest_int("n_estimators", 50, 300),
                    "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
                    "max_depth": trial.suggest_int("max_depth", 3, 12),
                    "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                    "gamma": trial.suggest_float("gamma", 0, 5),
                }

                return train(model_name, X, y, params)

        case "xgb-reg":
            def objective(trial):
                params = {
                    "n_estimators": trial.suggest_int("n_estimators", 50, 300),
                    "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
                    "max_depth": trial.suggest_int("max_depth", 3, 12),
                    "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                    "gamma": trial.suggest_float("gamma", 0, 5),
                }
                
                return train(model_name, X, y, params)
         
        case "rf-cls":
            def objective(trial):
                params = {
                    "n_estimators": trial.suggest_int("n_estimators", 50, 300),
                    "max_depth": trial.suggest_int("max_depth", 1, 30),  # 깊이를 1에서 30으로 조정
                    "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),  # 최소 샘플 분할 수
                    "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),  # 최소 리프 샘플 수
                }

                return train(model_name, X, y, params)
        
        case "rf-reg":
            def objective(trial):
                params = {
                    "n_estimators": trial.suggest_int("n_estimators", 50, 300),
                    "max_depth": trial.suggest_int("max_depth", 1, 30),  # 깊이를 1에서 30으로 조정
                    "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),  # 최소 샘플 분할 수
                    "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),  # 최소 리프 샘플 수
                }

                return train(model_name, X, y, params)

        case _:
            raise ValueError(f"지원하지 않는 모델 이름: {model_name}")
    
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="minimize", sampler=sampler)
    study.optimize(objective, n_trials=50)
    return study.best_params, study.best_value

### deposit_group 학습 및 예측

In [44]:
match selected_model:
    case "xgb": model_name = "xgb-cls"
    case "rf": model_name = "rf-cls"

In [45]:
if opt:
    best_params, _ = optuna_train(model_name, X, y) # 26m
else:
    best_params = {'n_estimators': 269, 'max_depth': 24, 'min_samples_split': 2, 'min_samples_leaf': 3}

# valid_logloss = cv_train(model_name, X, y, best_params)
# print(f"Valid logloss = {valid_logloss}")

Valid logloss = 0.10395885405874736


In [46]:
best_params

{'n_estimators': 269,
 'max_depth': 24,
 'min_samples_split': 2,
 'min_samples_leaf': 3}

In [47]:
match selected_model:
    case "xgb":
        best_model = xgb.XGBClassifier(**best_params, random_state=42, device="cuda", use_label_encoder=False, n_jobs=-1)
    case "rf":
        best_model = RandomForestClassifier(**best_params, random_state=42, n_jobs=-1)

best_model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=24, max_leaves=None,
              min_child_weight=None, min_samples_leaf=3, min_samples_split=2,
              missing=nan, monotone_constraints=None, multi_strategy=None,
              n_estimators=269, n_jobs=-1, ...)

In [48]:
predicted_groups = best_model.predict(test_data)
test_data['predicted_group'] = predicted_groups

### deposit 예측

In [49]:
match selected_model:
    case "xgb": model_name = "xgb-reg"
    case "rf": model_name = "rf-reg"

In [50]:
opt = True

In [51]:
group_params = {
        0: {'n_estimators': 299,
        'learning_rate': 0.07528019634863661,
        'max_depth': 12,
        'subsample': 0.9826861644413183,
        'colsample_bytree': 0.6361974955396621,
        'gamma': 0.13938848002312465},
    1: {'n_estimators': 276,
        'learning_rate': 0.15579191199373718,
        'max_depth': 12,
        'subsample': 0.909150931054429,
        'colsample_bytree': 0.8709809907337003,
        'gamma': 3.936332525239126},
    2: {'n_estimators': 282,
        'learning_rate': 0.05353956138863308,
        'max_depth': 12,
        'subsample': 0.9346361282442123,
        'colsample_bytree': 0.9739714687453176,
        'gamma': 4.082676340052684},
    3: {'n_estimators': 164,
        'learning_rate': 0.08351992311654627,
        'max_depth': 12,
        'subsample': 0.9598482300597749,
        'colsample_bytree': 0.8213376377326619,
        'gamma': 0.13938848002312465},
    4: {'n_estimators': 128,
        'learning_rate': 0.1224960449028662,
        'max_depth': 8,
        'subsample': 0.882895691571187,
        'colsample_bytree': 0.7384358080293545,
        'gamma': 1.3643518616482488},
    5: {'n_estimators': 60,
        'learning_rate': 0.0195312280618686,
        'max_depth': 8,
        'subsample': 0.604197860374508,
        'colsample_bytree': 0.9606755064214969,
        'gamma': 3.8599892180273407},
    6: {'n_estimators': 104,
        'learning_rate': 0.1408633637803759,
        'max_depth': 7,
        'subsample': 0.763889446515016,
        'colsample_bytree': 0.8858320716284691,
        'gamma': 0.9091078632943758},
    7: {'n_estimators': 172,
        'learning_rate': 0.164450647273469,
        'max_depth': 4,
        'subsample': 0.7642767983066922,
        'colsample_bytree': 0.8095967005933605,
        'gamma': 0.8290811474680035}
}

In [52]:
def train_regressors_per_group(train_data):
    group_models = {}
    group_params = {}
    group_scores = {}
    for group in tqdm(train_data['deposit_group'].unique(), desc="Training models per group"):
        group_data = train_data[train_data['deposit_group'] == group]
        X_group = group_data[selected_cols]
        y_group = group_data['deposit']

        # 모델 훈련
        if opt:
            best_params, best_value = optuna_train(model_name, X_group, y_group) # 26m
            group_params[group] = best_params
            group_scores[group] = best_value
        else:
            best_params = group_params[group]
            score = train(model_name, X, y, best_params)
            group_scores[group] = score
        
        match selected_model:
            case "xgb": model = xgb.XGBRegressor(**best_params, random_state=42, device="cuda", n_jobs=-1)
            case "rf": model = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)
        model.fit(X_group, y_group)
        
        # 각 그룹에 해당하는 모델 저장
        group_models[group] = model
        
    return group_models, group_params, group_scores

In [53]:
# 4. `deposit_group` 별로 회귀 모델 훈련
group_models, group_params, group_scores = train_regressors_per_group(train_data)

Training models per group:   0%|          | 0/8 [00:00<?, ?it/s][I 2024-10-22 14:44:55,329] A new study created in memory with name: no-name-6ddedfb8-f1ee-4bc2-9d61-c986220e732b


[I 2024-10-22 14:44:56,362] Trial 0 finished with value: 820.4625073939987 and parameters: {'n_estimators': 144, 'learning_rate': 0.19063571821788408, 'max_depth': 10, 'subsample': 0.7993292420985183, 'colsample_bytree': 0.5780093202212182, 'gamma': 0.7799726016810132}. Best is trial 0 with value: 820.4625073939987.
[I 2024-10-22 14:44:56,687] Trial 1 finished with value: 855.0481701351514 and parameters: {'n_estimators': 64, 'learning_rate': 0.1745734676972377, 'max_depth': 9, 'subsample': 0.8540362888980227, 'colsample_bytree': 0.5102922471479012, 'gamma': 4.8495492608099715}. Best is trial 0 with value: 820.4625073939987.
[I 2024-10-22 14:44:57,015] Trial 2 finished with value: 1088.0218646090386 and parameters: {'n_estimators': 258, 'learning_rate': 0.05034443102887247, 'max_depth': 4, 'subsample': 0.5917022549267169, 'colsample_bytree': 0.6521211214797689, 'gamma': 2.6237821581611893}. Best is trial 0 with value: 820.4625073939987.
[I 2024-10-22 14:44:57,789] Trial 3 finished with

In [54]:
group_params

{0: {'n_estimators': 299,
  'learning_rate': 0.07528019634863661,
  'max_depth': 12,
  'subsample': 0.9826861644413183,
  'colsample_bytree': 0.6361974955396621,
  'gamma': 0.13938848002312465},
 1: {'n_estimators': 276,
  'learning_rate': 0.15579191199373718,
  'max_depth': 12,
  'subsample': 0.909150931054429,
  'colsample_bytree': 0.8709809907337003,
  'gamma': 3.936332525239126},
 2: {'n_estimators': 282,
  'learning_rate': 0.05353956138863308,
  'max_depth': 12,
  'subsample': 0.9346361282442123,
  'colsample_bytree': 0.9739714687453176,
  'gamma': 4.082676340052684},
 3: {'n_estimators': 164,
  'learning_rate': 0.08351992311654627,
  'max_depth': 12,
  'subsample': 0.9598482300597749,
  'colsample_bytree': 0.8213376377326619,
  'gamma': 0.13938848002312465},
 4: {'n_estimators': 128,
  'learning_rate': 0.1224960449028662,
  'max_depth': 8,
  'subsample': 0.882895691571187,
  'colsample_bytree': 0.7384358080293545,
  'gamma': 1.3643518616482488},
 5: {'n_estimators': 60,
  'learni

In [55]:
group_scores

{0: 791.1157044955512,
 1: 3933.867696401091,
 2: 9990.004436592932,
 3: 15025.984248481782,
 4: 21288.49692622951,
 5: 19038.638671875,
 6: 9334.74107142857,
 7: 66495.765625}

In [56]:
counts = train_data.groupby('deposit_group')['deposit'].count()
scores = sum(score * counts[group] for group, score in group_scores.items())
total_count = counts.sum()
mean_score = scores / total_count

print(f"Mean MAE: {mean_score:.4f}")

Mean MAE: 4002.1930


In [57]:
def predict_per_group(test_data, group_models):
    # 예측값을 저장할 배열 초기화
    y_pred = np.zeros(len(test_data))
    
    # 그룹별로 데이터 분리 후 예측
    for group, model in group_models.items():
        group_data = test_data[test_data['predicted_group'] == group]
        X_group = group_data[selected_cols]
        
        # 각 그룹에 대해 예측
        if len(X_group) > 0:  # 해당 그룹에 데이터가 있는 경우만 예측
            y_pred_group = model.predict(X_group)
            y_pred[test_data['predicted_group'] == group] = y_pred_group

    return y_pred

In [58]:
y_test_pred = predict_per_group(test_data, group_models)

In [59]:
group_params = {int(k): v for k, v in group_params.items()}
wandb.log({
    "features": selected_cols,
    "model": selected_model,
    "params": best_params,
    "group_params": group_params,
    # "Valid logloss": valid_logloss,
    "Valid MAE": mean_score,
    "Optuna": opt
})
wandb.finish()

Valid MAE,▁
Optuna,True
Valid MAE,4002.19304
model,xgb


# Inference

In [52]:
sample_submission["deposit"] = y_test_pred

In [53]:
sample_submission

,index,deposit
0,0,20960.099609
1,1,6969.701172
2,2,5169.057129
3,3,5253.494141
4,4,6446.043945
...,...,...
150167,150167,38083.503906
150168,150168,44446.449219
150169,150169,40524.941406
150170,150170,37148.191406


In [54]:
sample_submission.to_csv("output.csv", index=False)